# Warsztat 3: Unity Catalog Governance & Integracje BI/ML

**Czas trwania:** 90 minut 
**Cel:** Praktyczne wykorzystanie Unity Catalog do zarządzania bezpieczeństwem i uprawnieniami, eksploracja data lineage, integracja z narzędziami BI (Power BI Direct Lake) oraz ML (MLflow, Feature Store).

---

## Cele warsztatowe

- Tworzenie katalogów i schematów w Unity Catalog
- Zarządzanie uprawnieniami: GRANT / REVOKE
- Eksploracja data lineage i audit logs
- Przygotowanie Gold Layer dla integracji BI
- Konfiguracja połączenia Power BI Direct Lake
- Podstawy MLflow: tracking eksperymentów
- Wykorzystanie Feature Store dla ML

---

## 📚 Konsolidowane zagadnienia

- Unity Catalog: Metastore, Catalogs, Schemas, Securable Objects
- Governance: Privileges, RBAC, Row-Level Security
- Lineage & Audit: end-to-end lineage tracking
- BI Integrations: Power BI Direct Lake
- ML Integrations: MLflow experiments, Feature Store

---

## Inicjalizacja środowiska

In [ ]:
%run ../../00_setup

---

## Część 1: Unity Catalog Setup

### Zadanie 1.1: Tworzenie katalogów i schematów

**Cel:** Utworzyć strukturę Unity Catalog dla projektu KION.

**Instrukcje:**
1. Utwórz katalog `kion_catalog` (jeśli nie istnieje)
2. Utwórz schematy: `bronze`, `silver`, `gold`
3. Sprawdź strukturę w Catalog Explorer

**TODO:** Uzupełnij SQL do tworzenia katalogów i schematów

In [ ]:
# TODO: Utwórz katalog kion_catalog
spark.sql("""
 CREATE CATALOG IF NOT EXISTS ___
""")

# TODO: Użyj utworzonego katalogu
spark.sql("USE CATALOG ___")

# TODO: Utwórz schematy bronze, silver, gold
for schema_name in [___, ___, ___]:
 spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
 print(f"Schemat {schema_name} utworzony")

### Zadanie 1.2: Weryfikacja struktury katalogów

**Cel:** Sprawdzić utworzoną strukturę Unity Catalog.

**TODO:** Wyświetl wszystkie schematy w katalogu

In [ ]:
# TODO: Pokaż wszystkie schematy w katalogu kion_catalog
spark.sql("___ SCHEMAS IN kion_catalog").display()

---

## Część 2: Zarządzanie uprawnieniami (GRANT / REVOKE)

### Zadanie 2.1: Przyznawanie uprawnień

**Cel:** Zrozumieć mechanizmy RBAC w Unity Catalog.

**Instrukcje:**
1. Przyznaj uprawnienia SELECT na schemacie `gold` dla grupy `data_analysts`
2. Przyznaj uprawnienia MODIFY na schemacie `silver` dla grupy `data_engineers`
3. Sprawdź aktywne uprawnienia

**TODO:** Uzupełnij polecenia GRANT

In [ ]:
# Przykład: Przyznaj uprawnienia SELECT dla data_analysts na gold schema
# TODO: Uzupełnij brakujące części

try:
 spark.sql("""
 GRANT ___ ON SCHEMA kion_catalog.gold TO `data_analysts`
 """)
 print("Uprawnienia SELECT przyznane dla data_analysts na gold")
except Exception as e:
 print(f"Uwaga: {e}")

# TODO: Przyznaj uprawnienia MODIFY dla data_engineers na silver schema
try:
 spark.sql("""
 GRANT ___ ON SCHEMA kion_catalog.silver TO `___`
 """)
 print("Uprawnienia MODIFY przyznane dla data_engineers na silver")
except Exception as e:
 print(f"Uwaga: {e}")

### Zadanie 2.2: Sprawdzanie uprawnień

**Cel:** Zweryfikować przyznane uprawnienia.

**TODO:** Wyświetl uprawnienia dla schematu gold

In [ ]:
# TODO: Wyświetl uprawnienia dla schematu gold
spark.sql("SHOW ___ ON SCHEMA kion_catalog.gold").display()

---

## Część 3: Data Lineage & Audit

### Zadanie 3.1: Przygotowanie danych do lineage

**Cel:** Utworzyć pipeline Bronze → Silver → Gold i zbadać lineage.

**Instrukcje:**
1. Załaduj dane do Bronze
2. Przetwórz do Silver z transformacjami
3. Utwórz agregaty w Gold
4. Sprawdź lineage w Catalog Explorer

**TODO:** Uzupełnij pipeline

In [ ]:
from pyspark.sql.functions import col, current_timestamp, sum as _sum, count, round as _round

# Bronze Layer: Wczytanie danych z Databricks Volume
volume_path = "/Volumes/main/default/kion_data"

bronze_customers = (
 spark.read
 .format("csv")
 .option("header", "true")
 .option("inferSchema", "true")
 .load(f"{volume_path}/customers/customers.csv")
 .withColumn("ingest_timestamp", current_timestamp())
)

# TODO: Zapisz do Bronze w Unity Catalog
bronze_customers.write.format("delta").mode("overwrite").saveAsTable("kion_catalog.___.customers_bronze")
print("Bronze layer: customers_bronze utworzona (z Volume)")

In [ ]:
# Silver Layer: Czyszczenie i transformacje
silver_customers = (
 spark.table("kion_catalog.bronze.customers_bronze")
 .filter(col("customer_id").isNotNull())
 .dropDuplicates(["customer_id"])
 .withColumn("processed_timestamp", current_timestamp())
)

# TODO: Zapisz do Silver
silver_customers.write.format("___").mode("___").saveAsTable("kion_catalog.silver.customers_silver")
print("Silver layer: customers_silver utworzona")

In [ ]:
# Wczytaj zamówienia z Databricks Volume
volume_path = "/Volumes/main/default/kion_data"

# Wczytaj dane zamówień z Volume (JSON)
orders_df = spark.read.json(f"{volume_path}/orders/orders_batch.json")

# Wczytaj dane produktów (do wzbogacenia)
products_df = spark.read.parquet(f"{volume_path}/products/products.parquet")

# Wzbogać zamówienia o produkty
enriched_orders = (
 orders_df
 .join(products_df, "product_id", "left")
 .select(
 "order_id",
 "customer_id",
 F.col("order_date").cast("timestamp").alias("order_date"),
 "status",
 F.col("total_amount").alias("amount")
 )
)

# TODO: Zapisz orders do Bronze
enriched_orders.withColumn("ingest_timestamp", F.current_timestamp()).write.format("delta").mode("overwrite").saveAsTable("kion_catalog.bronze.orders_bronze")
print("Bronze layer: orders_bronze utworzona (z Volume)")

In [ ]:
# Silver Layer: Przetwórz orders
silver_orders = (
 spark.table("kion_catalog.bronze.orders_bronze")
 .filter(col("status") == "completed")
 .withColumn("processed_timestamp", current_timestamp())
)

# TODO: Zapisz do Silver
silver_orders.write.format("delta").mode("overwrite").saveAsTable("kion_catalog.___.orders_silver")
print("Silver layer: orders_silver utworzona")

In [ ]:
# Gold Layer: Agregaty biznesowe - Customer Summary
# TODO: Uzupełnij agregaty (suma zamówień, liczba zamówień, średnia wartość)

gold_customer_summary = (
 spark.table("kion_catalog.silver.orders_silver")
 .groupBy("customer_id")
 .agg(
 ___(("amount")).alias("total_spent"),
 ___("order_id").alias("order_count"),
 ___((___(("amount")), 2)).alias("avg_order_value")
 )
)

# Połącz z danymi klientów
gold_customer_enriched = (
 gold_customer_summary
 .join(
 spark.table("kion_catalog.silver.customers_silver").select("customer_id", "customer_name", "country"),
 "customer_id",
 "left"
 )
)

# TODO: Zapisz do Gold
gold_customer_enriched.write.format("delta").mode("overwrite").saveAsTable("kion_catalog.___.customer_summary")
print("Gold layer: customer_summary utworzona")

gold_customer_enriched.display()

### Zadanie 3.2: Eksploracja Data Lineage

**Cel:** Sprawdzić end-to-end lineage w Catalog Explorer.

**Instrukcje:**
1. Przejdź do Catalog Explorer w Databricks UI
2. Znajdź tabelę `kion_catalog.gold.customer_summary`
3. Kliknij zakładkę **Lineage**
4. Zbadaj zależności: Bronze → Silver → Gold

**Pytania do rozważenia:**
- Jakie tabele są używane jako źródło dla Gold?
- Czy lineage pokazuje wszystkie transformacje?
- Jak można wykorzystać lineage w audytach i compliance?

---

## Część 4: Integracja BI - Power BI Direct Lake

### Zadanie 4.1: Przygotowanie Gold Layer dla Power BI

**Cel:** Przygotować dane w formacie optymalnym dla Power BI Direct Lake.

**Instrukcje:**
1. Optymalizuj tabelę Gold (OPTIMIZE, ZORDER)
2. Upewnij się, że kolumny mają właściwe typy danych
3. Dodaj komentarze do kolumn (dla dokumentacji)

**TODO:** Optymalizuj tabelę Gold

In [ ]:
# TODO: Optymalizuj tabelę Gold dla Power BI
spark.sql("""
 OPTIMIZE kion_catalog.gold.customer_summary
 ZORDER BY (___)
""")

print("Tabela customer_summary zoptymalizowana dla Power BI")

In [ ]:
# TODO: Dodaj komentarze do kolumn (dla dokumentacji)
spark.sql("""
 ALTER TABLE kion_catalog.gold.customer_summary 
 ALTER COLUMN customer_id COMMENT 'Unikalny identyfikator klienta'
""")

spark.sql("""
 ALTER TABLE kion_catalog.gold.customer_summary 
 ALTER COLUMN total_spent COMMENT '___'
""")

print("Komentarze do kolumn dodane")

### Zadanie 4.2: Konfiguracja połączenia Power BI

**Cel:** Zrozumieć konfigurację Power BI Direct Lake.

**Instrukcje (teoretyczne - wymaga Power BI Desktop):**
1. Otwórz Power BI Desktop
2. Wybierz **Get Data → Azure Databricks**
3. Wprowadź:
 - **Server hostname**: [workspace-url]
 - **HTTP Path**: [cluster-http-path]
 - **Catalog**: kion_catalog
 - **Schema**: gold
4. Wybierz tryb **DirectQuery** lub **Direct Lake** (jeśli dostępne)
5. Załaduj tabelę `customer_summary`

**Różnice:**
- **Direct Lake**: Natywne odczytywanie plików Delta (najszybsze)
- **DirectQuery**: Zapytania SQL do Databricks (real-time)
- **Import**: Kopiowanie danych do Power BI (najwolniejsze, ale offline)

---

## Część 5: Integracja ML - MLflow Tracking

### Zadanie 5.1: Utworzenie eksperymentu MLflow

**Cel:** Zarejestrować prosty eksperyment ML z wykorzystaniem MLflow.

**Instrukcje:**
1. Przygotuj dataset z Gold layer
2. Utwórz prosty model regresji (predykcja total_spent)
3. Zaloguj metryki i parametry w MLflow

**TODO:** Uzupełnij kod MLflow tracking

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# Przygotowanie danych
gold_df = spark.table("kion_catalog.gold.customer_summary").toPandas()

# Feature engineering - przykładowe cechy
X = gold_df[["order_count", "avg_order_value"]].fillna(0)
y = gold_df["total_spent"].fillna(0)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")

In [ ]:
# TODO: Uruchom eksperyment MLflow
mlflow.set_experiment("/Users/kion_training/customer_value_prediction")

with mlflow.start_run(run_name="linear_regression_baseline") as run:
 # TODO: Zaloguj parametry
 mlflow.log_param("model_type", "___")
 mlflow.log_param("test_size", ___)
 
 # Trenuj model
 model = LinearRegression()
 model.fit(X_train, y_train)
 
 # Predykcja
 y_pred = model.predict(X_test)
 
 # Oblicz metryki
 mse = mean_squared_error(y_test, y_pred)
 r2 = r2_score(y_test, y_pred)
 
 # TODO: Zaloguj metryki
 mlflow.log_metric("mse", ___)
 mlflow.log_metric("r2", ___)
 
 # TODO: Zaloguj model
 mlflow.sklearn.log_model(model, "model")
 
 print(f"Run ID: {run.info.run_id}")
 print(f"MSE: {mse:.2f}")
 print(f"R2 Score: {r2:.4f}")

### Zadanie 5.2: Eksploracja eksperymentów w MLflow UI

**Cel:** Przeglądać zalogowane eksperymenty w UI.

**Instrukcje:**
1. Przejdź do zakładki **Experiments** w Databricks UI
2. Znajdź eksperyment `/Users/kion_training/customer_value_prediction`
3. Sprawdź zalogowane metryki (MSE, R2)
4. Porównaj różne runy (jeśli uruchomisz model wielokrotnie)

**Pytania do rozważenia:**
- Jakie metryki są najważniejsze dla tego przypadku?
- Jak można zautomatyzować tracking w pipeline'ach produkcyjnych?
- Jak wykorzystać MLflow Model Registry?

---

## Część 6: Feature Store (podstawy)

### Zadanie 6.1: Utworzenie Feature Table

**Cel:** Zarejestrować cechy w Databricks Feature Store.

**Instrukcje:**
1. Przygotuj feature table z customer features
2. Zarejestruj w Feature Store
3. Wykorzystaj cechy w modelu ML

**TODO:** Uzupełnij kod Feature Store

In [ ]:
from databricks import feature_store
from databricks.feature_store import feature_table

# Inicjalizacja Feature Store Client
fs = feature_store.FeatureStoreClient()

# TODO: Przygotuj feature table
customer_features = (
 spark.table("kion_catalog.gold.customer_summary")
 .select(
 "customer_id",
 "total_spent",
 "order_count",
 "avg_order_value"
 )
)

# TODO: Utwórz feature table w Feature Store
try:
 fs.create_table(
 name="kion_catalog.gold.customer_features",
 primary_keys=["___"],
 df=customer_features,
 description="Customer features for ML models"
 )
 print("Feature table utworzona: kion_catalog.gold.customer_features")
except Exception as e:
 print(f"Feature table już istnieje lub błąd: {e}")
 # Jeśli już istnieje, zaktualizuj
 fs.write_table(
 name="kion_catalog.gold.customer_features",
 df=customer_features,
 mode="overwrite"
 )
 print("Feature table zaktualizowana")

### Zadanie 6.2: Wykorzystanie Feature Store w modelu

**Cel:** Odczytać cechy z Feature Store i użyć w modelu.

**TODO:** Załaduj cechy z Feature Store

In [ ]:
# TODO: Odczytaj feature table z Feature Store
features_df = fs.read_table(name="kion_catalog.gold.___")

features_df.display()

print("Cechy załadowane z Feature Store")

**Korzyści z Feature Store:**
- Centralne repozytorium cech
- Wersjonowanie cech
- Śledzenie lineage cech → modele
- Udostępnianie cech między zespołami
- Spójność cech: training vs inference

---

## Część 7: Podsumowanie i Best Practices

### Zrealizowane cele

- Utworzenie katalogów i schematów w Unity Catalog
- Zarządzanie uprawnieniami: GRANT / REVOKE
- Budowa pipeline Bronze → Silver → Gold z lineage
- Eksploracja data lineage w Catalog Explorer
- Przygotowanie danych dla Power BI Direct Lake
- Tracking eksperymentów w MLflow
- Wykorzystanie Feature Store

---

### Best Practices: Unity Catalog

1. **Hierarchia:** Metastore → Catalog → Schema → Table/View
2. **Principle of Least Privilege:** Przyznawaj minimalne wymagane uprawnienia
3. **Separation of Concerns:** Bronze (raw), Silver (clean), Gold (business)
4. **Lineage tracking:** Używaj Unity Catalog dla automatycznego lineage
5. **Dokumentacja:** Dodawaj komentarze do tabel i kolumn

---

### Best Practices: BI Integrations

1. **Direct Lake:** Najszybszy tryb dla Power BI (jeśli dostępny)
2. **Optymalizacja:** OPTIMIZE + ZORDER przed udostępnieniem do BI
3. **Denormalizacja:** Gold layer często wymaga denormalizacji dla wydajności
4. **Typy danych:** Upewnij się, że typy są kompatybilne z BI tools
5. **Incremental refresh:** Wykorzystaj partycjonowanie dla odświeżania przyrostowego

---

### Best Practices: ML Integrations

1. **MLflow tracking:** Zawsze loguj metryki, parametry, artifacts
2. **Model Registry:** Rejestruj modele produkcyjne w MLflow Model Registry
3. **Feature Store:** Centralizuj cechy dla spójności training/inference
4. **Reproducibility:** Wersjonuj dane, kod, modele
5. **Monitoring:** Śledź model drift i data drift w produkcji

---

## 🎓 Zadanie dodatkowe (opcjonalne)

### Rozszerzone Governance

1. **Row-Level Security:** Zaimplementuj row-level security w Unity Catalog
 - Utwórz widok z filtrem na podstawie `current_user()`
 - Przetestuj dostęp dla różnych użytkowników

2. **Column Masking:** Ukryj wrażliwe dane (np. email, phone)
 - Wykorzystaj `CASE WHEN is_member('sensitive_group')` do maskowania

3. **Delta Sharing:** Udostępnij dane zewnętrznie
 - Utwórz share w Unity Catalog
 - Udostępnij tabelę Gold dla zewnętrznego odbiorcy

4. **Audit Logging:** Zbadaj audit logs
 - Sprawdź `system.access.audit` w Unity Catalog
 - Zidentyfikuj aktywność użytkowników na tabelach

---

## 📚 Materiały dodatkowe

- [Unity Catalog Documentation](https://docs.databricks.com/data-governance/unity-catalog/index.html)
- [Power BI Direct Lake](https://learn.microsoft.com/en-us/power-bi/enterprise/directlake-overview)
- [MLflow Documentation](https://mlflow.org/docs/latest/index.html)
- [Databricks Feature Store](https://docs.databricks.com/machine-learning/feature-store/index.html)
- [Delta Sharing Protocol](https://delta.io/sharing/)

---

**Koniec warsztatu!** 